In [ ]:
import sqlite3

from datetime import datetime, timedelta
from prettytable import PrettyTable

conn = sqlite3.connect('books.db')
conn.execute('''CREATE TABLE IF NOT EXISTS books
             (id INTEGER,
             name TEXT PRIMARY KEY NOT NULL,
             is_available BOOLEAN NOT NULL,
             borrower_name TEXT )''')

conn.execute('''CREATE TABLE IF NOT EXISTS users
             (
             mobile_number TEXT PRIMARY KEY UNIQUE NOT NULL,
             name TEXT ,
             is_active BOOLEAN NOT NULL,
             borrowed_books_num INT ,
             borrowed_books_name TEXT ,
             date_of_borrowing TEXT )''')



def add_new_user():
    mobile_number = input("Enter User Mobile Number: ")

    while len(mobile_number) != 10 or not mobile_number.isdigit():
        mobile_number = input("Invalid Mobile Number. Please Enter a 10-digit Number: ")
    name = input("Enter name: ")
    while not name.isalpha():
        name = input("Invalid Name. Please Enter a Valid Name : ")
    is_active = 1
    borrowed_books_num = 0
    borrowed_books_name = ""
    date_of_borrowing = "none"
    try:
        conn.execute(f"INSERT INTO users (mobile_number, name, is_active, borrowed_books_num, borrowed_books_name, date_of_borrowing) VALUES ('{mobile_number}', '{name}', {is_active}, {borrowed_books_num}, '{borrowed_books_name}', '{date_of_borrowing}')")
        conn.commit()
        print(f"New user: {name}. Mobile: {mobile_number} Added Successfully")
    except sqlite3.IntegrityError:
        print("User already Exists.")
        

def enter_to_exist_user():
    mobile_number = input("Enter Mobile Number: ")
    cursor = conn.execute(f"SELECT * FROM users WHERE mobile_number='{mobile_number}'")
    row = cursor.fetchone()
    if row:
        print(f"User found: {row[1]} - {row[0]}")
        select=int(input("What Do you Want To Do with User :Type the number of option\n1.Borrow Book\n2. Return Book\n"))
        
        if select == 1:
            borrow_book(mobile_number)
        elif select == 2:
            return_book()
        else:
            print("Please Type a Correct Option Number ")
    else:
        print("User not found")

def borrow_book(mobile_number):
    book_name = input("Enter a Book Name: ")
    book_name = book_name.upper()
    cursor = conn.execute(f"SELECT * FROM books WHERE name='{book_name}'")
    row = cursor.fetchone()
    if row:
        if row[2]:
            cursor2 = conn.execute(f"SELECT * FROM users WHERE mobile_number='{mobile_number}'")
            row2 = cursor2.fetchone()
            if row2[2] ==1  and row2[3] < 2:
                
                if row2[4] != "":
                    borrowed_books_name = f"{row2[4]} ,{book_name}"
                else:
                    borrowed_books_name = book_name
                date_of_borrowing = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                conn.execute(f"UPDATE books SET is_available=0, borrower_name='{row2[0]}' WHERE name='{book_name}'")
                conn.execute(f"UPDATE users SET borrowed_books_num=borrowed_books_num+1, borrowed_books_name='{borrowed_books_name}', date_of_borrowing='{date_of_borrowing }' WHERE mobile_number='{mobile_number}'")
                conn.commit()
                print("Book Borrowed Successfully!")
            else:
                print("Cannot Borrow Book. User is not active or has already borrowed 2 books.")
        else:
            print("Book Not Available")
    else:
        print("Book not found")

def return_book():
    book_name = input("Enter the book name: ")
    book_name=book_name.upper()
    cursor = conn.execute(f"SELECT * FROM books WHERE name='{book_name}'")
    row = cursor.fetchone()
    if row:
        if  row[2] == 0 : # means already borrowed
            borrower_name = row[3]
            conn.execute(f"UPDATE books SET is_available=1, borrower_name='none' WHERE name='{book_name}'")
            conn.execute(f"UPDATE users SET borrowed_books_num = borrowed_books_num - 1, borrowed_books_name = " ", date_of_borrowing = ' ' WHERE mobile_number = '{borrower_name} '")

            conn.commit()
            print("Book Returned Successfully!")
        else:
            print("Book Already Available")
    else:
        print("Book not found")

def show_borrowed_books():
        
    cursor= conn.execute('SELECT books.name, users.name ,users.mobile_number  FROM books INNER JOIN users ON books.borrower_name = users.mobile_number')
    results = cursor.fetchall()
    table = PrettyTable(['Book Name', 'Borrower Name', 'Borrower Mobile'])
    for row in results:
        table.add_row(row)
    print(table)

        

print("Welcome To Book Borrowing System")
while True:
        print("-"*50)
        print("\nSelect an option: Type number of option")
        print("1. Add New User")
        print("2. Borrow or Return a Book")
        print("3. Show Borrowed Books")
        print("-"*50)
        option = input()
        if option not in ['1', '2', '3']:
            print("Invalid Input. Please Enter a Number Between 1 and 3.")
        else:
            if option == '1':
                add_new_user()

            elif option == '2':
                enter_to_exist_user()

            else: 
                show_borrowed_books()
 

conn.close()

Welcome To Book Borrowing System
--------------------------------------------------

Select an option: Type number of option
1. Add New User
2. Borrow or Return a Book
3. Show Borrowed Books
--------------------------------------------------
1
Enter User Mobile Number: 1111111111
Enter name: Dalal
New user: Dalal. Mobile: 1111111111 Added Successfully
--------------------------------------------------

Select an option: Type number of option
1. Add New User
2. Borrow or Return a Book
3. Show Borrowed Books
--------------------------------------------------
1
Enter User Mobile Number: 1111111111
Enter name: sara
User already Exists.
--------------------------------------------------

Select an option: Type number of option
1. Add New User
2. Borrow or Return a Book
3. Show Borrowed Books
--------------------------------------------------
